In [1]:
import pandas as pd
#from pandas_datareader import DataReader
import numpy as np
from pandas import json_normalize
#import matplotlib.pyplot as plt
#import seaborn as sns
import streamlit as st
import base64
import plotly.express as px
from PIL import Image
import plotly.graph_objects as go

##stock packages
#from yahoo_fin import stock_info as si
import yfinance as yfin
#from get_all_tickers import get_tickers as gt

## web crawling packages
import time
import requests
#import lxml.html as lh
from lxml import html
import csv

## selenium package
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

# yfinance not working

In [105]:
stock = "A"

In [106]:
ticker = yfin.Ticker(stock)

In [107]:
info = ticker.info
info_df = json_normalize(info)
info_df

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,companyOfficers,website,...,fromCurrency,fiveYearAvgDividendYield,fiftyTwoWeekLow,bid,tradeable,dividendYield,bidSize,dayHigh,regularMarketPrice,logo_url
0,95051,Healthcare,16500,"Agilent Technologies, Inc. provides applicatio...",Santa Clara,800 227 9770,CA,United States,[],http://www.agilent.com,...,None,0.82,93.87,150.31,False,0.0052,800,150.5,150.3,https://logo.clearbit.com/agilent.com


In [108]:
dividends = ticker.dividends
dividends

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [98]:
calendar = ticker.calendar
calendar

,Value
Earnings Date,2021-08-17 00:00:00
Earnings Average,0.99
Earnings Low,0.97
Earnings High,1.03
Revenue Average,1539440000
Revenue Low,1519000000
Revenue High,1575600000


In [99]:
quarterly_earnings = ticker.quarterly_earnings
quarterly_earnings

,Revenue,Earnings
Quarter,,
3Q2020,1261000000,199000000
4Q2020,1483000000,222000000
1Q2021,1548000000,288000000
2Q2021,1525000000,216000000


In [109]:
period = "3mo"

In [110]:
prices = ticker.history(period = period)
prices

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [92]:
msft = yfin.Ticker("MSFT")
msft.history(period="max")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
recommendations = ticker.recommendations
recommendations

,Firm,To Grade,From Grade,Action
Date,,,,
2012-02-16 06:01:00,Deutsche Bank,Buy,,main
2012-03-06 09:28:00,Morgan Stanley,Overweight,,init
2012-04-19 08:35:00,Stifel Nicolaus,Buy,,main
2012-05-15 06:14:00,Morgan Stanley,Overweight,,main
2012-05-15 08:07:00,Stifel Nicolaus,Buy,,main
...,...,...,...,...
2021-05-26 10:10:42,Keybanc,Overweight,,main
2021-05-26 10:20:35,SVB Leerink,Outperform,,main
2021-05-26 12:08:05,Wells Fargo,Overweight,,main


In [13]:
recommendations.groupby(['Action', 'To Grade']).size()

Action  To Grade    
down    Equal-Weight     1
        Hold             2
        Neutral          3
        Outperform       1
        Sell             1
init    Buy              2
        Equal-Weight     1
        Hold             2
        Neutral          4
        Outperform       3
        Overweight       5
main    Buy             37
        Equal-Weight     8
        Hold             3
        Neutral         11
        Outperform      14
        Overweight      31
up      Buy              5
        Outperform       4
        Overweight       2
        Strong Buy       1
dtype: int64

In [17]:
recommendations_agg = pd.DataFrame(recommendations.groupby(['To Grade', 'Firm']).size()).reset_index()
recommendations_agg.columns = ['Grade', 'Firm', 'Count of Analyst']
recommendations_agg

,Grade,Firm,Count of Analyst
0,Buy,BTIG Research,2
1,Buy,Bank of America,6
2,Buy,Citigroup,7
3,Buy,Deutsche Bank,4
4,Buy,Evercore ISI Group,1
5,Buy,Goldman Sachs,4
6,Buy,ISI Group,1
7,Buy,Jefferies,7
8,Buy,Longbow Research,1
9,Buy,Needham,1


## Crawling Stock Analysis Page

In [7]:
from selenium import webdriver
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

web = "https://stockanalysis.com/stocks/"
driver = webdriver.Chrome()
#driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get(web)
sel = Select(driver.find_element_by_xpath('//select[@name="perpage"]'))
sel.select_by_visible_text("10000")

time.sleep(5)

ticker_list = []
company_name_list = []
industry_list = []

## starting to find elements
ticker = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[1]/a')
for a in range(len(ticker)):
    ticker_list.append(ticker[a].text)

company_name = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[2]')
for b in range(len(company_name)):
    company_name_list.append(company_name[b].text)

industry = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[3]')
for c in range(len(industry)):
    industry_list.append(industry[c].text)

## Creating dataframes so I can join this all together
ticker_df = pd.DataFrame(ticker_list)
company_name_df = pd.DataFrame(company_name_list)
industry_df = pd.DataFrame(industry_list)
big_df = pd.concat([ticker_df, company_name_df, industry_df], axis=1)

KeyboardInterrupt: 

In [47]:
## creating a couple new columns to create a link and full searchable stock name plus ticker to help with app search.
big_df.columns = ['ticker', 'company_name', 'industry']
big_df['full'] = big_df['ticker'] + " - " + big_df['company_name']
big_df['url'] = "https://stockanalysis.com/stocks/" + big_df['ticker'] + "/"
big_df['full'] = big_df['full'].astype('str')
big_df['ticker'] = big_df.ticker.str.strip()

## create csv to use for streamlit app
big_df.to_csv('data/tickers_only.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720 entries, 0 to 6719
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   full    6720 non-null   object
 1   url     6720 non-null   object
dtypes: object(2)
memory usage: 105.1+ KB


In [1]:
#split = df["full"].str.split("-", n = 1, expand = True)
#df['ticker'] = split[0]
#df['company_name'] = split[1]
#df.head()

## Crawl Stock Analysis Website to collect most recent month trends and 3 month trends to look at overall industry trends

In [ ]:
//*[@id="main"]/div[2]/div[1]/div/div[1]/ul/li[3]/button

In [21]:
web = "https://stockanalysis.com/stocks/aapl/"
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get(web)
sel = Select(driver.find_element_by_xpath('//*[@id="main"]/div[2]/div[1]/div/div[1]/ul/li[3]/button'))
sel.select_by_visible_text("1M")

UnexpectedTagNameException: Message: Select only works on <select> elements, not on <button>


In [ ]:
web = "https://stockanalysis.com/stocks/aapl/"
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get(web)
sel = Select(driver.find_element_by_xpath('//button[@name="perpage"]'))
sel.select_by_visible_text("1M")

time.sleep(5)

ticker_list = []
company_name_list = []
industry_list = []

## starting to find elements
ticker = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[1]/a')
for a in range(len(ticker)):
    ticker_list.append(ticker[a].text)

company_name = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[2]')
for b in range(len(company_name)):
    company_name_list.append(company_name[b].text)

industry = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[3]')
for c in range(len(industry)):
    industry_list.append(industry[c].text)

## news from yahoo finance. We can only pull 3 articles :(

In [120]:
ticker = 'AAPL'

In [121]:
pt1 = 'https://finance.yahoo.com/quote/'
pt2 = '?p='
pt3 = '&.tsrc=fin-srch'
url = pt1 + ticker + pt2 + ticker + pt3
print(url)

https://finance.yahoo.com/quote/AAPL?p=AAPL&.tsrc=fin-srch


In [123]:
pt1 = 'https://finance.yahoo.com/quote/'
pt2 = '?p='
pt3 = '&.tsrc=fin-srch'
url = pt1 + ticker + pt2 + ticker + pt3

r = requests.get(url)
data = html.fromstring(r.text)

# Xpath
alldata =[]

#time.sleep(2)

for i in data.xpath("//li[contains(@class,'js-stream-content Pos(r)')]"):
    stock = i.xpath('div/div/div[2]/h3/a/text()')
    #url = i.xpath('a/@href')[0]
    #year = i.xpath('span[2]/text()')   
    #print(title, url, year)
    alldata.append([stock])
    
len(alldata)

3

In [119]:
pt1 = 'https://finance.yahoo.com/quote/'
pt2 = '?p='
pt3 = '&.tsrc=fin-srch'
url = pt1 + ticker + pt2 + ticker + pt3

r = requests.get(url)
data = html.fromstring(r.text)

# Xpath
alldata =[]

for i in data.xpath("//li[@class='js-stream-content Pos(r)']"):
    stock = i.xpath('div/div/div[2]/h3/a/text()')
    #url = i.xpath('a/@href')[0]
    #year = i.xpath('span[2]/text()')   
    #print(title, url, year)
    alldata.append([stock])
    
len(alldata)

3

In [118]:
alldata

[[['Dow Jones Futures: Market Rally Looks To Extend Gains With 5 Stocks In Buy Zones; Tesla, Nio Story Time Over']],
 [['Is GM Stock A Buy? General Motors Rebounds Bullishly Amid Deeper FedEx Ties']],
 [["Elon Musk Tweets Tesla Will Be Bigger Than Apple 'In A Few Months', Then Deletes Tweet"]]]

## Google News

In [5]:
"""
## example urls
## https://news.google.com/search?q=AAPL&hl=en-US&gl=US&ceid=US%3Aen
## https://news.google.com/search?q=TSLA&hl=en-US&gl=US&ceid=US%3Aen
"""

'\n## example urls\n##https://news.google.com/search?q=AAPL&hl=en-US&gl=US&ceid=US%3Aen\n##https://news.google.com/search?q=TSLA&hl=en-US&gl=US&ceid=US%3Aen\n'

In [ ]:
//*[@id="yDmH0d"]/c-wiz/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[1]/div/article/h3/a

In [17]:
first_part = 'https://news.google.com/search?q='
ticker_desc = 'AAPL'
second_part = '&hl=en-US&gl=US&ceid=US%3Aen'

url = first_part + ticker_desc + second_part
#driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver.get(url)

request = requests.get(url)
#r = requests.get('https://www.marketwatch.com/investing/stock/AAPL')
data = html.fromstring(request.text)

# Xpath
alldata =[]

for i in data.xpath("//div[contains(@jslog,'93789')]"):
    stock = i.xpath('div/h3/a/text()')
    url = i.xpath('div/h3/a/@href')
    #year = i.xpath('span[2]/text()')   
    #print(title, url, year)
    alldata.append([stock, url])
    
len(alldata)

InvalidSelectorException: Message: invalid selector: The result of the xpath expression "//*[@id="yDmH0d"]/c-wiz/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[1]/div/article/h3/a/@href" is: [object Attr]. It should be an element.
  (Session info: chrome=92.0.4515.159)


In [16]:
article_url_df

,0
0,AAPL How COVID Affects Apple (AAPL)


## lets try marketwatch

In [61]:
ticker_desc = 'AAPL'

In [62]:
url = 'https://www.marketwatch.com/investing/stock/' + ticker_desc
request = requests.get(url)
#r = requests.get('https://www.marketwatch.com/investing/stock/AAPL')
data = html.fromstring(request.text)

# Xpath
alldata =[]

for i in data.xpath("//div[contains(@class,'element element--article')]"):
    stock = i.xpath('div/h3/a/text()')
    url = i.xpath('div/h3/a/@href')
    #year = i.xpath('span[2]/text()')   
    #print(title, url, year)
    alldata.append([stock, url])
    
len(alldata)

48

In [63]:
df = pd.DataFrame(alldata)
df.head(2)

,0,1
0,[\n \n ...,[https://www.marketwatch.com/articles/alphabet...
1,[\n \n ...,[https://www.marketwatch.com/articles/the-case...


In [64]:
df[0]=df[0].str[0]
df[1]=df[1].str[0]
df = df.rename(columns = {0: 'title', 1: 'url'})
df.head(2)

,title,url
0,\n \n ...,https://www.marketwatch.com/articles/alphabet-...
1,\n \n ...,https://www.marketwatch.com/articles/the-case-...


In [65]:
df['title'] = df['title'].astype('str')
df['title'] = df.title.str.strip()
df.head()

,title,url
0,Alphabet Stock Can Be Had for a Discount. Here...,https://www.marketwatch.com/articles/alphabet-...
1,There’s a New Growth Area in Tech. Pay Attenti...,https://www.marketwatch.com/articles/the-case-...
2,The Meme Stock Trade Is Far From Over. What In...,https://www.marketwatch.com/articles/the-meme-...
3,"Look Out Alphabet, Amazon, and Apple: Biden Ta...",https://www.marketwatch.com/articles/biden-big...
4,"Apple Inc. stock rises Friday, outperforms market",https://www.marketwatch.com/story/apple-inc-st...


In [66]:
df = df[df["title"].str.contains("nan")==False]
df.head(20)

,title,url
0,Alphabet Stock Can Be Had for a Discount. Here...,https://www.marketwatch.com/articles/alphabet-...
1,There’s a New Growth Area in Tech. Pay Attenti...,https://www.marketwatch.com/articles/the-case-...
2,The Meme Stock Trade Is Far From Over. What In...,https://www.marketwatch.com/articles/the-meme-...
3,"Look Out Alphabet, Amazon, and Apple: Biden Ta...",https://www.marketwatch.com/articles/biden-big...
4,"Apple Inc. stock rises Friday, outperforms market",https://www.marketwatch.com/story/apple-inc-st...
5,A Tesla ‘death cross’ has appeared for the fir...,https://www.marketwatch.com/story/a-tesla-deat...
6,Falling Bond Yields Used to Be Good for the Na...,https://www.marketwatch.com/articles/bond-yiel...
7,Apple Inc. stock underperforms Thursday when c...,https://www.marketwatch.com/story/apple-inc-st...
8,3 Key Things to Know About the Google App Stor...,https://www.marketwatch.com/articles/google-pl...
9,"China is cracking down on its own tech giants,...",https://www.marketwatch.com/story/china-is-cra...


In [67]:
for article in df['url']:
    print(article)

https://www.marketwatch.com/articles/alphabet-stock-discount-51625588865?mod=mw_quote_news
https://www.marketwatch.com/articles/the-case-for-network-monitoring-tools-and-unpacking-apples-app-store-risk-51625867520?mod=mw_quote_news
https://www.marketwatch.com/articles/the-meme-stock-trade-is-far-from-over-what-investors-need-to-know-51625875247?mod=mw_quote_news
https://www.marketwatch.com/articles/biden-big-tech-executive-order-51625853683?mod=mw_quote_news
https://www.marketwatch.com/story/apple-inc-stock-rises-friday-outperforms-market-01625862708-4160a0b12b13?mod=mw_quote_news
https://www.marketwatch.com/story/a-tesla-death-cross-is-coming-the-first-in-more-than-2-years-11625771674?mod=mw_quote_news
https://www.marketwatch.com/articles/bond-yields-stock-market-nasdaq-tech-51625759540?mod=mw_quote_news
https://www.marketwatch.com/story/apple-inc-stock-underperforms-thursday-when-compared-to-competitors-01625776307-23730b68e2ca?mod=mw_quote_news
https://www.marketwatch.com/articles/g

In [69]:
df_test = df.head(5)

In [71]:
import time

article_text = []

for article in df_test['url']:
    data = html.fromstring(requests.get(article).text)

    time.sleep(3)

    for j in data.xpath("//div[@id='js-article__body']"):
        text = j.xpath("p/text()")
        article_text.append([text])

len(article_text)

5

In [72]:
article_text

[[['A quirk in the relationship between the two publicly traded classes of \n        ',
   '\n       stock has resulted in the nonvoting shares, whose ticker is GOOG, trading at a premium of 3% to the voting stock, whose ticker is GOOGL.',
   '\xa0For investors bullish on the outlook for the search giant, the class A voting stock may be the better bet. Alphabet’s nonvoting class C shares (ticker: GOOG) were up $4.21, to $2,578.59, on Tuesday, a premium of about $77 a share to the class A voting shares (GOOGL), which were down $3.33, to $2,501.82.']],
 [['One of the challenges of modern enterprise computing is keeping track of what’s going on inside your network. Cutting-edge information-technology systems are a messy brew of public clouds, private clouds, old-school data centers, third-party apps, edge computing, and mobile workers. Keeping tabs on what’s working—and what isn’t—is a gigantic challenge. The good news for investors is that the result is an enormous emerging market.']],
 

## crawling stock option chains

"""

Stock Option Data from - "https://www.nasdaq.com/market-activity/stocks/"

Definitions and Terms - from "https://www.nerdwallet.com/article/investing/options-trading-definitions"

Strike: The price you'd pay or receive if you exercised the option.

Contract name: Just like stocks have ticker symbols, options contracts have option symbols with letters and numbers that correspond to the details in a contract. In a real option chain, the company’s ticker symbol would come before the contract name.

Last: The price that was paid or received the last time the option was traded.

Bid: The price a buyer is willing to pay for the option. If you're selling an option, this is the premium you'd receive for the contract.

Ask: The price a seller is willing to accept for the option. If you want to buy an option, this is the premium you'd pay.

Change: The price change since the previous trading day’s close, also expressed in percentage terms.

Volume: The number of contracts traded that day.

Open interest: The number of options contracts currently in play.


In the money: This refers to an option that has intrinsic value — when the relationship between stock price in the open market and the strike price favors the options contract owner. When the stock price is higher than the strike price, that’s good news for the owner of a call option. A put option is in the money if the stock price is lower than the strike price.

Out of the money: When there's no financial benefit to exercising the option, it's called out of the money. Practically speaking, an out-of-the-money option makes buying or selling shares at the strike price less lucrative than buying or selling on the open market. A call option is out of the money if the stock price is lower than the strike price. A put option is out of the money when the stock price is higher than the strike price.

At the money: When the stock price is roughly equal to the strike price, an option is considered at the money. Basically, it’s a wash.

"""

In [2]:
import requests
import json
import lxml.html as lh
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from pandas import json_normalize
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import re

In [4]:
#url = "https://api.nasdaq.com/api/quote/AAPL/option-chain?assetclass=stocks&limit=60"
url = "https://api.nasdaq.com/api/quote/TSLA/option-chain?assetclass=stocks&fromdate=all&todate=undefined&excode=oprac&callput=callput&money=all&type=all"

payload={}
headers = {
  'User-Agent': 'PostmanRuntime/7.28.4',
  'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate, br',
  'Content-Length': '1970',
  'Connection': 'keep-alive'
}

response = requests.request("GET", url, headers=headers, data=payload)

response_text = json.loads(response.text)
json_normalize(response_text)

,message,data.totalRecord,data.lastTrade,data.table.headers.expiryDate,data.table.headers.c_Last,data.table.headers.c_Change,data.table.headers.c_Bid,data.table.headers.c_Ask,data.table.headers.c_Volume,data.table.headers.c_Openinterest,...,data.filterlist.type.filtertype,data.filterlist.type.filterid,data.filterlist.type.filter,data.filterlist.fromdate.filtertitle,data.filterlist.fromdate.filtertype,data.filterlist.fromdate.filterid,data.filterlist.fromdate.filter,status.rCode,status.bCodeMessage,status.developerMessage
0,None,2589,"LAST TRADE: $805.72 (AS OF OCT 12, 2021)",Exp. Date,Last,Change,Bid,Ask,Volume,Open Int.,...,dropdown,type,"[{'label': 'All (Types)', 'value': 'all'}, {'l...",Date,dropdown,fromdate,"[{'label': 'October 2021', 'value': '2021-10-0...",200,None,None


In [5]:
price = json_normalize(response_text['data'])
price = pd.DataFrame(price['lastTrade'], columns=['lastTrade'])
price[['last', 'trade', 'price', 'as', 'of', 'sep', 'day', 'year']]= price["lastTrade"].str.split(" ", expand = True)
price_new = price['price'].str[1:][0]
price_new

'805.72'

In [6]:
json_normalize(response_text['data']['filterlist']['callput'], 'filter')

,label,value
0,Calls & Puts,callput
1,Calls,call
2,Puts,put


In [111]:
option_data = json_normalize(response_text['data']['table'], 'rows')
print(option_data.shape)

(2532, 18)


In [112]:
option_data = option_data.drop(columns = ['drillDownURL'])
option_data['expirygroup'] = option_data['expirygroup'].replace('', np.nan).ffill()
option_data['expirygroup'] = pd.to_datetime(option_data['expirygroup'])
option_data = option_data.dropna(subset = ['strike'])
#option_data = option_data.tail(-1)
option_data

,expirygroup,expiryDate,c_Last,c_Change,c_Bid,c_Ask,c_Volume,c_Openinterest,c_colour,strike,p_Last,p_Change,p_Bid,p_Ask,p_Volume,p_Openinterest,p_colour
1,2021-09-24,Sep 24,663.30,23.70,665.00,681.65,4,1384,True,100.00,0.01,--,--,0.01,--,963,False
2,2021-09-24,Sep 24,562.35,15.20,565.75,583.25,14,2,True,200.00,0.01,--,--,0.01,31,596,False
3,2021-09-24,Sep 24,462.45,8.25,465.75,483.25,1,7,True,300.00,0.01,--,--,0.01,--,678,False
4,2021-09-24,Sep 24,365.80,15.10,365.00,384.50,9,25,True,400.00,0.01,--,--,0.01,20,2553,False
5,2021-09-24,Sep 24,344.20,9.30,354.50,374.00,4,16,True,410.00,0.01,--,--,0.01,2,339,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,2024-01-19,Jan 19,82.19,7.89,80.10,88.00,131,74,False,1400.00,730.00,--,682.00,701.50,--,13,True
2528,2024-01-19,Jan 19,--,--,75.50,85.00,--,--,False,1425.00,--,--,704.00,723.50,--,--,True
2529,2024-01-19,Jan 19,68.21,--,73.00,82.00,--,152,False,1450.00,750.10,--,726.50,745.50,--,1,True
2530,2024-01-19,Jan 19,72.75,7.99,70.00,79.50,2,4,False,1475.00,--,--,748.50,767.50,--,--,True


In [113]:
price_new = float(price_new)
type(price_new)

float

In [114]:
calls = option_data[['expirygroup', 'c_Last', 'c_Change', 'c_Bid', 'c_Ask', 'c_Volume', 'c_Openinterest', 'c_colour', 'strike']].copy()
calls = calls.rename(columns = {'c_Last': 'Last', 'c_Change': 'Change', 'c_Bid': 'Bid', 'c_Ask': 'Ask', 'c_Volume': 'Volume', 'c_Openinterest': 'Openinterest', 'c_colour': 'colour'})
calls['type'] = "Call"
calls['strike'] = calls['strike'].astype(float)
calls['money_category'] = np.where(calls['strike'] <= price_new, "In the Money", "Out of the Money")

puts = option_data[['expirygroup', 'p_Last', 'p_Change', 'p_Bid', 'p_Ask', 'p_Volume', 'p_Openinterest', 'p_colour', 'strike']].copy()
puts = calls.rename(columns = {'p_Last': 'Last', 'p_Change': 'Change', 'p_Bid': 'Bid', 'p_Ask': 'Ask', 'p_Volume': 'Volume', 'p_Openinterest': 'Openinterest', 'p_colour': 'colour'})
puts['type'] = "Put"
puts['strike'] = puts['strike'].astype(float)
puts['money_category'] = np.where(puts['strike'] >= price_new, "In the Money", "Out of the Money")

option_data_new = calls.append(puts)
option_data_new['current_price'] = price_new
option_data_new

,expirygroup,Last,Change,Bid,Ask,Volume,Openinterest,colour,strike,type,money_category,current_price
1,2021-09-24,663.30,23.70,665.00,681.65,4,1384,True,100.0,Call,In the Money,774.39
2,2021-09-24,562.35,15.20,565.75,583.25,14,2,True,200.0,Call,In the Money,774.39
3,2021-09-24,462.45,8.25,465.75,483.25,1,7,True,300.0,Call,In the Money,774.39
4,2021-09-24,365.80,15.10,365.00,384.50,9,25,True,400.0,Call,In the Money,774.39
5,2021-09-24,344.20,9.30,354.50,374.00,4,16,True,410.0,Call,In the Money,774.39
...,...,...,...,...,...,...,...,...,...,...,...,...
2527,2024-01-19,82.19,7.89,80.10,88.00,131,74,False,1400.0,Put,In the Money,774.39
2528,2024-01-19,--,--,75.50,85.00,--,--,False,1425.0,Put,In the Money,774.39
2529,2024-01-19,68.21,--,73.00,82.00,--,152,False,1450.0,Put,In the Money,774.39
2530,2024-01-19,72.75,7.99,70.00,79.50,2,4,False,1475.0,Put,In the Money,774.39


In [115]:
option_data_new = option_data_new.replace('--', 0)
option_data_new

,expirygroup,Last,Change,Bid,Ask,Volume,Openinterest,colour,strike,type,money_category,current_price
1,2021-09-24,663.30,23.70,665.00,681.65,4,1384,True,100.0,Call,In the Money,774.39
2,2021-09-24,562.35,15.20,565.75,583.25,14,2,True,200.0,Call,In the Money,774.39
3,2021-09-24,462.45,8.25,465.75,483.25,1,7,True,300.0,Call,In the Money,774.39
4,2021-09-24,365.80,15.10,365.00,384.50,9,25,True,400.0,Call,In the Money,774.39
5,2021-09-24,344.20,9.30,354.50,374.00,4,16,True,410.0,Call,In the Money,774.39
...,...,...,...,...,...,...,...,...,...,...,...,...
2527,2024-01-19,82.19,7.89,80.10,88.00,131,74,False,1400.0,Put,In the Money,774.39
2528,2024-01-19,0,0,75.50,85.00,0,0,False,1425.0,Put,In the Money,774.39
2529,2024-01-19,68.21,0,73.00,82.00,0,152,False,1450.0,Put,In the Money,774.39
2530,2024-01-19,72.75,7.99,70.00,79.50,2,4,False,1475.0,Put,In the Money,774.39


In [116]:
option_data_new['Change'] = option_data_new['Change'].astype(float)
option_data_new['Bid'] = option_data_new['Bid'].astype(float)
option_data_new['Ask'] = option_data_new['Ask'].astype(float)
option_data_new['Volume'] = option_data_new['Volume'].astype(float)
option_data_new['Openinterest'] = option_data_new['Openinterest'].astype(float)
option_data_new['strike'] = option_data_new['strike'].astype(float)
option_data_new['current_price'] = option_data_new['current_price'].astype(float)
option_data_new.head()

,expirygroup,Last,Change,Bid,Ask,Volume,Openinterest,colour,strike,type,money_category,current_price
1,2021-09-24,663.30,23.70,665.00,681.65,4.0,1384.0,True,100.0,Call,In the Money,774.39
2,2021-09-24,562.35,15.20,565.75,583.25,14.0,2.0,True,200.0,Call,In the Money,774.39
3,2021-09-24,462.45,8.25,465.75,483.25,1.0,7.0,True,300.0,Call,In the Money,774.39
4,2021-09-24,365.80,15.10,365.00,384.50,9.0,25.0,True,400.0,Call,In the Money,774.39
5,2021-09-24,344.20,9.30,354.50,374.00,4.0,16.0,True,410.0,Call,In the Money,774.39


### testing visuals

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
There is no [win32] chromedriver for browser 93.0.4577 in cache
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\rober\.wdm\drivers\chromedriver\win32\93.0.4577.63]


In [ ]:
/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div/div/div[1]/div/div[4]/div/button[6]
/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div/div/div[1]/div/div[4]/div/button[7]
/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div/div/div[1]/div/div[4]/div/button[8]

In [2]:
web = "https://www.nasdaq.com/market-activity/stocks/aapl/option-chain"
#driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(web)



buttons_list = []
call_list = []


buttons = driver.find_elements_by_xpath('//div[@class = "pagination__pages"]')
for p in range(len(buttons)):
    buttons_list.append(buttons[p].text)

call = driver.find_elements_by_xpath('//table/tbody/tr')
for l in range(len(call)):
    call_list.append(call[l].text)

print("done")



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\rober\.wdm\drivers\chromedriver\win32\93.0.4577.63]


done


In [3]:
buttons_list

['1']

In [4]:
df = pd.DataFrame(call_list, columns = ['response'])
df['exp_date'] = df['response'].str[:6]
df['response'] = df['response'].str[6:].str.strip()
df[['call_last', 'call_change', 'call_bid', 'call_ask', 'call_volume', 'call_open_int', 'strike', 'put_last', 'put_change', 'put_bid', 'put_ask', 'put_volume', 'put_open_int']]= df["response"].str.split(" ", expand = True)
df = df.tail(-1)
df = df.drop(columns = ['response'])
df

,exp_date,call_last,call_change,call_bid,call_ask,call_volume,call_open_int,strike,put_last,put_change,put_bid,put_ask,put_volume,put_open_int
1,Sep 24,13.12,0.22,12.90,13.05,229,310,134.00,0.01,--,--,0.01,213,6876
2,Sep 24,11.99,0.08,11.90,12.80,645,941,135.00,0.01,-0.01,--,0.01,506,6394
3,Sep 24,10.85,--,10.90,11.05,390,584,136.00,0.01,--,--,0.01,116,3911
4,Sep 24,10.02,0.12,9.50,10.05,29,245,137.00,0.01,--,--,0.01,197,4088
5,Sep 24,8.84,-0.08,8.50,9.10,77,911,138.00,0.01,--,--,0.01,915,10133
6,Sep 24,7.84,0.02,7.90,8.10,38,537,139.00,0.01,-0.01,--,0.01,1414,5564
7,Sep 24,6.97,0.07,6.90,7.05,1251,2885,140.00,0.01,-0.01,--,0.01,1959,13267
8,Sep 24,6.00,0.05,5.50,6.05,242,1859,141.00,0.01,-0.01,--,0.01,2137,8365
9,Sep 24,4.95,0.05,4.50,5.85,1411,4670,142.00,0.01,-0.03,--,0.01,1536,9404
10,Sep 24,3.90,--,3.50,4.75,2257,4431,143.00,0.01,-0.05,--,0.01,13946,11248


In [56]:
web = "https://www.nasdaq.com/market-activity/stocks/plug/option-chain"
#driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(web)



headers_list = []
exp_date_list = []
call_last_list = []
call_change_list = []
call_bid_list = []
call_ask_list = []
call_volume_list = []
call_open_int_list = []
strike_list = []
put_last_list = []
put_change_list = []
put_bid_list = []
put_ask_list = []
put_volume_list = []
put_open_int_list = []


headers = driver.find_elements_by_xpath('//table/thead/tr[2]/th')
for p in range(len(headers)):
    headers_list.append(headers[p].text)

exp_date = driver.find_elements_by_xpath('//table/tbody/tr/td[2]')
for l in range(len(exp_date)):
    exp_date_list.append(exp_date[l].text)

## calls
call_last = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_last)):
    call_last_list.append(call_last[l].text)

call_change = driver.find_elements_by_xpath('//table/tbody/tr/td[4]')
for l in range(len(call_change)):
    call_change_list.append(call_change[l].text)

call_bid = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_bid)):
    call_bid_list.append(call_bid[l].text)

call_ask = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_ask)):
    call_ask_list.append(call_ask[l].text)

call_volume = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_volume)):
    call_volume_list.append(call_volume[l].text)

call_open_int = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_open_int)):
    call_open_int_list.append(call_open_int[l].text)

strike = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(strike)):
    strike_list.append(strike[l].text)

## puts
call_last = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_last)):
    call_last_list.append(call_last[l].text)

call_change = driver.find_elements_by_xpath('//table/tbody/tr/td[4]')
for l in range(len(call_change)):
    call_change_list.append(call_change[l].text)

call_bid = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_bid)):
    call_bid_list.append(call_bid[l].text)

call_ask = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_ask)):
    call_ask_list.append(call_ask[l].text)

call_volume = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_volume)):
    call_volume_list.append(call_volume[l].text)

call_open_int = driver.find_elements_by_xpath('//table/tbody/tr/td[3]')
for l in range(len(call_open_int)):
    call_open_int_list.append(call_open_int[l].text)


print(len(headers_list))
print(len(exp_date_list))
print(len(call_last_list))



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\rober\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


14
18


In [57]:
print(type(headers_list))
headers_list

<class 'list'>


['Exp. Date',
 'Last',
 'Change',
 'Bid',
 'Ask',
 'Volume',
 'Open Int.',
 'Strike',
 'Last',
 'Change',
 'Bid',
 'Ask',
 'Volume',
 'Open Int.']

In [62]:
#row_list = row_list.replace("\n", ", ")
print(type(exp_date))
exp_date_list

<class 'list'>


ValueError: Shape of passed values is (18, 1), indices imply (18, 14)

In [21]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [45]:
df = pd.DataFrame(row_list, columns = ['response'])
#df[['date','last', 'change']] = df.Name.str.split("_",expand=True)
#df['response'] = df['response'].str.replace("--", '')
#df['response'] = df['response'].str.replace("", ', ')
#df[['First','Last']] = df.Name.str.split("_",expand=True)
df

,response
0,"September 17, 2021"
1,Sep 17 1.96 -- -- -- -- -- 24.50 0.01 -- -- --...
2,Sep 17 1.48 -- -- -- -- 3 25.00 0.01 -- -- -- ...
3,Sep 17 0.99 -- -- -- -- 1 25.50 0.01 -- -- -- ...
4,Sep 17 0.50 -- -- -- -- -- 26.00 0.02 -- -- --...
5,Sep 17 0.04 -- -- -- -- 3539 26.50 0.11 -- -- ...
6,Sep 17 0.01 -- -- -- -- 6927 27.00 0.53 -- -- ...
7,Sep 17 0.01 -- -- -- -- 2561 27.50 1.23 -- -- ...
8,Sep 17 0.01 -- -- -- -- 2869 28.00 1.50 -- -- ...
9,Sep 17 0.02 -- -- -- -- 1316 28.50 2.18 -- -- ...


## scraping stock option chain data

In [3]:
ticker_df = pd.read_csv("tickers_only.csv")
tickers = ticker_df['ticker']
tickers

0          A
1         AA
2        AAC
3       AACG
4       AADI
        ... 
5931     ZWS
5932      ZY
5933    ZYME
5934    ZYNE
5935    ZYXI
Name: ticker, Length: 5936, dtype: object

In [13]:
i = "AAPL"

base1 = "https://api.nasdaq.com/api/quote/"
base2 = "/option-chain?assetclass=stocks&fromdate=all&todate=undefined&excode=oprac&callput=callput&money=all&type=all"
url = base1 + str(i) + base2

payload={}

#headers = {'server':'Kestrel', 'Accept':'application/json, text/plain, */*','Accept-Encoding': 'gzip, deflate, br', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
headers = {'User-Agent': 'PostmanRuntime/7.28.4','Accept': '*/*','Accept-Encoding': 'gzip, deflate, br'}

response = requests.get(url, headers=headers, data=payload)

response_text = json.loads(response.text)
pd.DataFrame(response_text, columns = ['option_data'])

,option_data


In [18]:
import time

#ticker = "AAPL"

response_dict = {"ticker":[],"option_data":[]}

for i in tickers[:5]:

    time.sleep(2)

    base1 = "https://api.nasdaq.com/api/quote/"
    base2 = "/option-chain?assetclass=stocks&fromdate=all&todate=undefined&excode=oprac&callput=callput&money=all&type=all"
    url = base1 + str(i) + base2

    payload={}

    #headers = {'server':'Kestrel', 'Accept':'application/json, text/plain, */*','Accept-Encoding': 'gzip, deflate, br', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    headers = {'User-Agent': 'PostmanRuntime/7.28.4','Accept': '*/*','Accept-Encoding': 'gzip, deflate, br'}

    response = requests.get(url, headers=headers, data=payload)

    response_text = json.loads(response.text)
    response_dict['ticker'].append(i)
    response_dict['option_data'].append(response_text)

print(len(response_dict))

2


In [20]:
response_df = pd.DataFrame(response_dict)
response_df

,ticker,option_data
0,A,"{'data': {'totalRecord': 280, 'lastTrade': 'LA..."
1,AA,"{'data': {'totalRecord': 403, 'lastTrade': 'LA..."
2,AAC,"{'data': {'totalRecord': 0, 'lastTrade': None,..."
3,AACG,"{'data': {'totalRecord': 0, 'lastTrade': None,..."
4,AADI,"{'data': {'totalRecord': 0, 'lastTrade': None,..."


In [23]:
def gather_tickers():
    ##importing files needed for web app
    #ticker_selection = pd.read_csv('data/tickers_only.csv')
    #tickers = ticker_selection['ticker']
    
    response_dict = {"ticker":[],"option_data":[]}
    error_list = []
    
    for i in tickers[:5]:
        
        time.sleep(2)
        
        try:
            base1 = "https://api.nasdaq.com/api/quote/"
            base2 = "/option-chain?assetclass=stocks&fromdate=all&todate=undefined&excode=oprac&callput=callput&money=all&type=all"
            url = base1 + str(i) + base2

            payload={}

            #headers = {'server':'Kestrel', 'Accept':'application/json, text/plain, */*','Accept-Encoding': 'gzip, deflate, br', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
            headers = {'User-Agent': 'PostmanRuntime/7.28.4','Accept': '*/*','Accept-Encoding': 'gzip, deflate, br'}

            response = requests.get(url, headers=headers, data=payload)

            response_text = json.loads(response.text)
            response_dict['ticker'].append(i)
            response_dict['option_data'].append(response_text)
            print("Loaded Option Chain Data for:", i)

        except Exception as e:
            error_list.append([i, e])
            print("Error:", e)

        big_df = pd.DataFrame(response_dict)
        big_df.to_csv('option_chain_data.csv')

    return big_df

response_df = gather_tickers()
response_df

Loaded Option Chain Data for: A
Loaded Option Chain Data for: AA
Loaded Option Chain Data for: AAC
Loaded Option Chain Data for: AACG
Loaded Option Chain Data for: AADI


,ticker,option_data
0,A,"{'data': {'totalRecord': 280, 'lastTrade': 'LA..."
1,AA,"{'data': {'totalRecord': 403, 'lastTrade': 'LA..."
2,AAC,"{'data': {'totalRecord': 0, 'lastTrade': None,..."
3,AACG,"{'data': {'totalRecord': 0, 'lastTrade': None,..."
4,AADI,"{'data': {'totalRecord': 0, 'lastTrade': None,..."


## dividing tickers for jobs

In [5]:
ticker_df.head()

,Unnamed: 0,ticker,company_name,industry,full,url
0,0,A,Agilent Technologies,Life Sciences Tools & Services,A - Agilent Technologies,https://stockanalysis.com/stocks/A/
1,1,AA,Alcoa,Metals & Mining,AA - Alcoa,https://stockanalysis.com/stocks/AA/
2,2,AAC,Ares Acquisition,Shell Companies,AAC - Ares Acquisition,https://stockanalysis.com/stocks/AAC/
3,3,AACG,ATA Creativity Global,Diversified Consumer Services,AACG - ATA Creativity Global,https://stockanalysis.com/stocks/AACG/
4,4,AADI,Aadi Bioscience,Biotechnology,AADI - Aadi Bioscience,https://stockanalysis.com/stocks/AADI/


In [10]:
ticker_df['group'] = ticker_df.groupby(np.arange(len(ticker_df.index))//(len(ticker_df)/10),axis=0).ngroup()+1
ticker_df.head()

,Unnamed: 0,ticker,company_name,industry,full,url,group
0,0,A,Agilent Technologies,Life Sciences Tools & Services,A - Agilent Technologies,https://stockanalysis.com/stocks/A/,1
1,1,AA,Alcoa,Metals & Mining,AA - Alcoa,https://stockanalysis.com/stocks/AA/,1
2,2,AAC,Ares Acquisition,Shell Companies,AAC - Ares Acquisition,https://stockanalysis.com/stocks/AAC/,1
3,3,AACG,ATA Creativity Global,Diversified Consumer Services,AACG - ATA Creativity Global,https://stockanalysis.com/stocks/AACG/,1
4,4,AADI,Aadi Bioscience,Biotechnology,AADI - Aadi Bioscience,https://stockanalysis.com/stocks/AADI/,1


In [11]:
ticker_df.groupby('group').size()

group
1     594
2     594
3     593
4     594
5     594
6     593
7     594
8     593
9     594
10    593
dtype: int64

In [ ]:
group1 = []
group2 = []
group3 = []
group4 = []
group5 = []
group6 = []
group7 = []
group8 = []
group9 = []
group10 = []

for i in ticker_df:

    if ticker_df['group'] == 1:
        group1.append(i)
        elif word in negative_words:
            negative += 1
    if positive-negative > 0:
        positive_review_bl.append(sentence)
    elif positive-negative == 0:
        neutral_review_bl.append(sentence)
    else:
        negative_review_bl.append(sentence)
        
print(len(positive_review_bl))
print(len(negative_review_bl))
print(len(neutral_review_bl))

## Scrapping stock analysis statistics

In [150]:
url = "https://stockanalysis.com/_next/data/LOYm7LDXfcgxvDTmXVza9/stocks/tsla/company.json"

payload={}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

response_text = json.loads(response.text)

json_normalize(response_text['pageProps']['info'])

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))